In [ ]:
import pandas as pd
from vnstock import Finance
from IPython.display import display # Dùng để hiển thị bảng đẹp trong Jupyter
import time
import os


In [ ]:
symbol_file = '../data/raw/vietnam_stock_symbols.csv'


In [ ]:
# Tạo List danh sách các symbols cần lấy thông tin
df_symbols = pd.read_csv(symbol_file)
symbols_list = df_symbols['symbol'].tolist()
# symbols_list

In [ ]:
# Giúp hiển thị đầy đủ tất cả các cột (năm/quý)
pd.set_option('display.max_columns', None)
# Mở rộng chiều ngang để bảng không bị xuống dòng lung tung
pd.set_option('display.width', 1000)
# Định dạng số thực hiển thị 2 số lẻ cho gọn
pd.set_option('display.float_format', '{:,.2f}'.format)

In [ ]:
# Năm giữ lại dữ liệu
target_years = [2023, 2024, 2025]

In [ ]:
# Lọc dữ liệu theo năm
def filter_data_by_years(df, year_index_in_df, target_years = target_years):
    year_column = df.columns[year_index_in_df]
    # display(df) # uncomment để xem dữ liệu
    return df[df[year_column].isin(target_years)]

In [ ]:
# Lưu dataframe vào file
def save_to_csv_file(filename, df):
    df.to_csv(filename, index = False, encoding = 'utf-8-sig')

In [ ]:
# Bảng dự phòng, ghi thông tin từng bước trong quá trình duyệt, tránh trường hợp bị lỗi trước khi lưu vào fie chính thức
def save_one_by_one(filename, df):
    df.to_csv(
        filename,
        mode='a',       # ghi nối tiếp
        index=False,
        header=not os.path.exists(filename),  # chỉ ghi header khi file chưa tồn tại
        encoding='utf-8-sig'
    )

In [ ]:
# [optional] Báo cáo kết quả kinh doanh của một mã công ty
def report(df_income):
    print(f"BÁO CÁO KẾT QUẢ KINH DOANH ({symbol})")
    print("-" * 80)
    # Nếu chạy trong Jupyter Notebook, dùng display(df_income) sẽ đẹp hơn print
    try:
        display(df_income) 
    except:
        print(df_income)
    print("\n" + "="*80 + "\n")


# BÁO CÁO TÀI CHÍNH

In [ ]:
df_income_list = []
symbol_error_list = []
file_one_by_one = '../../data/raw/fa/income_one_by_one.csv'

for x in symbols_list: 
    try:
        if (len(x) < 3):
            continue
            
        finance = Finance(symbol = x, source = 'VCI')
        df_income = finance.income_statement(period='quarter', lang='vi')
        if df_income.empty:
            finance = Finance(symbol = x, source = 'TCBS')
            df_income = finance.income_statement(period='quarter', lang='vi')
            
        if df_income.empty:
            continue
                
        df_income = filter_data_by_years(df_income, 1, target_years)
        
        if not df_income.empty:
            df_income_list.append(df_income)
            save_one_by_one(file_one_by_one, df_income)

    except SystemExit as se:
        wait_time = 60
        time.sleep(wait_time)
    except Exception as e:
            symbol_error_list.append(x)
            # 6. BẮT LỖI CRASH (KeyError, Mất mạng, Lỗi API...)
            print(f"!!! LỖI NGHIÊM TRỌNG với mã {x}: {e}. Bỏ qua.")

df_income_official = pd.DataFrame()
if df_income_list:
    df_income_official  = pd.concat(df_income_list, ignore_index = True)    

In [ ]:
# Các mã không lấy được
print(f'Số mã không lấy được: {len(symbol_error_list)}')
print(f'Tên mã: {symbol_error_list}')


In [ ]:
filename = '../../data/raw/fa/baocaotaichinh.csv'
save_to_csv_file(filename, df_income_official)

# BẢNG CÂN ĐỐI KẾ TOÁN

In [ ]:
df_balance_list = [] 
symbol_error_list = []
file_one_by_one = '../../data/raw/fa/balance_one_by_one.csv'

for x in symbols_list: 
    try:
        if (len(x) < 3):
            continue
            
        # 1. THỬ TRUY XUẤT NGUỒN VCI 
        finance = Finance(symbol = x, source = 'VCI') 
        df_balance = finance.balance_sheet(period='quarter', lang='vi')

        # 2. THỬ LẠI VỚI NGUỒN TCBS NẾU DỮ LIỆU RỖNG
        if df_balance.empty:
            # SỬA LỖI 1: Thay symbol thành x
            finance = Finance(symbol = x, source = 'TCBS') 
            df_balance = finance.balance_sheet(period='quarter', lang='vi')
            
        if df_balance.empty:
            continue
                
        # 3. LỌC DỮ LIỆU
        df_balance = filter_data_by_years(df_balance, 1, target_years)
        
        # 4. LƯU KẾT QUẢ
        if not df_balance.empty:
            df_balance_list.append(df_balance) 
  
            save_one_by_one(file_one_by_one, df_balance) 

    except SystemExit as se:
        wait_time = 60
        time.sleep(wait_time)
    except Exception as e:
        symbol_error_list.append(x) 
        print(f"!!! LỖI NGHIÊM TRỌNG với mã {x}: {e}. Bỏ qua.")

# 5. TỔNG HỢP CUỐI CÙNG
df_balance_official = pd.DataFrame() 
if df_balance_list:
    df_balance_official = pd.concat(df_balance_list, ignore_index = True)

In [ ]:
# Các mã không lấy được
print(f'Số mã không lấy được: {len(symbol_error_list)}')
print(f'Tên mã: {symbol_error_list}')

In [ ]:
filename = '../../data/raw/fa/bangcandoiketoan.csv'
save_to_csv_file(filename, df_balance_official)

# BẢNG LƯU CHUYỂN TIỀN TỆ

In [ ]:
df_cashflow_list = [] # Danh sách để lưu các DataFrame Cash Flow thành công
symbol_error_list = []
file_one_by_one = '../../data/raw/fa/cashflow_one_by_one.csv' # Tên file để lưu từng báo cáo riêng lẻ

for x in symbols_list:
    try:
        if (len(x) < 3):
            continue
            
        # 1. THỬ TRUY XUẤT NGUỒN VCI 
        finance = Finance(symbol = x, source = 'VCI') 
        df_cashflow = finance.cash_flow(period='quarter', lang='vi') # <-- Lấy Cash Flow

        # 2. THỬ LẠI VỚI NGUỒN TCBS NẾU DỮ LIỆU RỖNG
        if df_cashflow.empty:
            finance = Finance(symbol = x, source = 'TCBS') 
            df_cashflow = finance.cash_flow(period='quarter', lang='vi') # <-- Thử lại Cash Flow
            
        if df_cashflow.empty:
            continue
                
        # 3. LỌC DỮ LIỆU
        df_cashflow = filter_data_by_years(df_cashflow, 1, target_years)
        
        # 4. LƯU KẾT QUẢ
        if not df_cashflow.empty:
            df_cashflow_list.append(df_cashflow) 
            save_one_by_one(file_one_by_one, df_cashflow) # Lưu file Cash Flow riêng lẻ

    except SystemExit as se:
        # Bắt lỗi hệ thống/mạng và tạm dừng
        wait_time = 60
        time.sleep(wait_time)
    except Exception as e:
        # Bắt các lỗi khác
        symbol_error_list.append(x) 
        print(f"!!! LỖI NGHIÊM TRỌNG với mã {x}: {e}. Bỏ qua.")


## Tổng hợp Kết quả Cuối Cùng
df_cashflow_official = pd.DataFrame() 
if df_cashflow_list:
    df_cashflow_official = pd.concat(df_cashflow_list, ignore_index = True)


In [ ]:
# Các mã không lấy được
print(f'Số mã không lấy được: {len(symbol_error_list)}')
print(f'Tên mã: {symbol_error_list}')


In [ ]:
filename = '../../data/raw/fa/bangluuchuyentiente.csv'
save_to_csv_file(filename, df_balance_official)
